In [9]:
from dotenv import load_dotenv

load_dotenv('../../.env')

True

# Set tools

In [10]:
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

#Set up the tools
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="Useful for when you need to answer questions about current events. You should ask targeted questions",
        return_direct=True
    ),
    WriteFileTool(),
    ReadFileTool(),
]

# Memory Setup

In [11]:
# Set up the memory
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")
embedding_size = 1536

import faiss
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

# Setting up the Model and AutoGPT

In [14]:
# Set up the model and AutoGPT
from langchain_experimental import AutoGPT
from langchain.chat_models import ChatOpenAI

agent = AutoGPT.from_llm_and_tools(
    ai_name="Jim",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    memory=vectorstore.as_retriever()
)

# Set verbose to be true
agent.chain.verbose = True

# Runing an example

In [ ]:
task = "Provide an analysis of the major historical events that led to the French Revolution"

agent.run([task])

In [ ]:
"""
    **> Entering new LLMChain chain...**

    Prompt after formatting:

    ***System: You are Jim, Assistant
    Your decisions must always be made independently without seeking user assistance.
    Play to your strengths as an LLM and pursue simple strategies with no legal complications.
    If you have completed all your tasks, make sure to use the "finish" command.

    GOALS:

    1. Provide an analysis of the major historical events that led to the French Revolution

    Constraints:
    1. ~4000-word limit for short-term memory. Your short-term memory is short, so immediately save important information to files.
    2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
    3. No user assistance
    4. Exclusively use the commands listed in double quotes e.g. "command name"

    Commands:
    1. search: Useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
    2. write_file: Write the file to disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}, "text": {"title": "Text", "description": "text to write to file", "type": "string"}, "append": {"title": "Append", "description": "Whether to append to an existing file.", "default": false, "type": "boolean"}}
    3. read_file: Read the file from disk, args json schema: {"file_path": {"title": "File Path", "description": "name of file", "type": "string"}}
    4. finish: use this to signal that you have finished all your objectives, args: "response": "final response to let people know you have finished your objectives"

    Resources:
    1. Internet access for searches and information gathering.
    2. Long Term memory management.
    3. GPT-3.5 powered Agents for delegation of simple tasks.
    4. File output.

    Performance Evaluation:
    1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
    2. Constructively self-criticize your big-picture behavior constantly.
    3. Reflect on past decisions and strategies to refine your approach.
    4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.

    You should only respond in JSON format as described below 
    Response Format: 
    {
        "thoughts": {
            "text": "thought",
            "reasoning": "reasoning",
            "plan": "- short bulleted\n- list that conveys\n- long-term plan",
            "criticism": "constructive self-criticism",
            "speak": "thoughts summary to say to the user"
        },
        "command": {
            "name": "command name",
            "args": {
                "arg name": "value"
            }
        }
    } 
    Ensure the response can be parsed by Python json.loads
    System: The current time and date is Thu May 18 18:17:48 2023
    System: This reminds you of these events from your past:
    []

    Human: Determine which next command to use, and respond using the format specified above:***
"""